# 005: Cross Validate Models

In [1]:
import os
import sys

import numpy as np

sys.path.append("../")
from models import OrdinaryLeastSquares, LogisticRegression, LinearSVM, KNearestNeighbors, DecisionTree
from model_selection import cross_validation
from preprocessing import preprocess, preprocess_splits

%load_ext autoreload
%autoreload 2

def print_results(cv_results):
    print(f"Average F1-score: {np.mean(cv_results.f1_scores)*100:.1f}% ± {np.std(cv_results.f1_scores)*100:.1f}")
    print(f"Average F2-score: {np.mean(cv_results.f2_scores)*100:.1f}% ± {np.std(cv_results.f2_scores)*100:.1f}")
    print(f"Average AUC-ROC: {np.mean(cv_results.auc_rocs)*100:.1f}% ± {np.std(cv_results.auc_rocs)*100:.1f}")
    print(f" & {np.mean(cv_results.f1_scores)*100:.1f}±{np.std(cv_results.f1_scores)*100:.1f} & "
          f"{np.mean(cv_results.f2_scores)*100:.1f}±{np.std(cv_results.f2_scores)*100:.1f} & "
          f"{np.mean(cv_results.auc_rocs)*100:.1f}±{np.std(cv_results.auc_rocs)*100:.1f}")

In [ ]:
x_train, _, y_train, *_ = preprocess(one_hot_encoding=False, save_dir="../data/dataset_prep")
x_train.shape

In [3]:
train = np.load("../data/dataset_prep/train.npz")
x_train, y_train = train["x_train"], train["y_train"]
x_train.shape

(328135, 622)

In [4]:
model_settings = [
    #{"model_class": OrdinaryLeastSquares},
    #{"model_class": LogisticRegression},
    #{"model_class": LinearSVM},
    #{"model_class": KNearestNeighbors},
    {"model_class": DecisionTree},
]
for model in model_settings:
    print(f"Cross-validating model: {model['model_class'].__name__}")
    num_samples = int(1e4) if model['model_class'] != KNearestNeighbors else int(1e5)
    cv_results = cross_validation(x_train[:num_samples][:,:321], y_train[:num_samples], verbose=True, **model)
    print_results(cv_results)
    if num_samples != 1e6: break
    out_dir = "../results"
    os.makedirs(out_dir, exist_ok=True)
    out_path = os.path.join(out_dir, f"{model['model_class'].__name__}.txt")

    with open(out_path, "w") as f:
        f.write(f"Model: {model['model_class'].__name__}\n\n")
        f.write("cv_results repr:\n")
        f.write(repr(cv_results))
        f.write("\n\n")
        f.write("Aggregated metrics:\n")
        f.write(f"Average F1-score: {np.mean(cv_results.f1_scores)*100:.1f}% ± {np.std(cv_results.f1_scores)*100:.1f}\n")
        f.write(f"Average F2-score: {np.mean(cv_results.f2_scores)*100:.1f}% ± {np.std(cv_results.f2_scores)*100:.1f}\n")
        f.write(f"Average AUC-ROC: {np.mean(cv_results.auc_rocs)*100:.1f}% ± {np.std(cv_results.auc_rocs)*100:.1f}\n")
    

Cross-validating model: DecisionTree
Starting fold 1/5 with 8000 samples
depth=3, min_leaf=20 -> score=0.3585 thr=0.10
depth=3, min_leaf=50 -> score=0.3735 thr=0.10
depth=3, min_leaf=100 -> score=0.3851 thr=0.10
depth=3, min_leaf=200 -> score=0.3860 thr=0.10
depth=4, min_leaf=20 -> score=0.3891 thr=0.09
depth=4, min_leaf=50 -> score=0.3982 thr=0.18
depth=4, min_leaf=100 -> score=0.4038 thr=0.12
depth=4, min_leaf=200 -> score=0.3972 thr=0.12
depth=5, min_leaf=20 -> score=0.3741 thr=0.04
depth=5, min_leaf=50 -> score=0.3829 thr=0.23
depth=5, min_leaf=100 -> score=0.3841 thr=0.17
depth=5, min_leaf=200 -> score=0.3972 thr=0.12
depth=6, min_leaf=20 -> score=0.3762 thr=0.21
depth=6, min_leaf=50 -> score=0.3871 thr=0.17
depth=6, min_leaf=100 -> score=0.3841 thr=0.17
depth=6, min_leaf=200 -> score=0.3972 thr=0.16
depth=8, min_leaf=20 -> score=0.3583 thr=0.05
depth=8, min_leaf=50 -> score=0.3717 thr=0.15
depth=8, min_leaf=100 -> score=0.3794 thr=0.10
depth=8, min_leaf=200 -> score=0.3972 thr=0.

KeyboardInterrupt: 

In [ ]:
& 41.8±0.2 & 48.7±0.7 & 85.4±0.4